# Retrieval Augmented Generation using generative-ai-hub-sdk and HANA vector search
<!-- description --> Using HANA vector store to store vector embeddings and using them in Retrieval Augmented Generation.

## Prerequisites

- Access to SAP AI core with sap extended plan.
- Have python3 installed in your system.
- Have generative-ai-hub-sdk installed in your system.

- ### Loading vector data from a csv file

Download the following ```csv``` file and save it in your system.

 [Download File](files/GRAPH_DOCU_QRC3.csv)

Execute the following python code in the same folder. This will load the data and store it in a data-frame.

In [13]:
import pandas as pd
df = pd.read_csv('GRAPH_DOCU_2503.csv', low_memory=False)
df.head(3)

,ID,L1,L2,L3,FILENAME,HEADER1,HEADER2,TEXT,VECTOR_STR
0,273,90,40,0,090-040-000-Appendix_C_-_GraphScript_Cheat_She...,Appendix C - GraphScript Cheat Sheet,Weighted Path Functions,<!--! subsection -->\r\n### WEIGHT \r\n```gra...,"[0.015699435,0.020284351,0.0003677337,-0.00413..."
1,52,60,20,30,060-020-030-Basic_Vertex_Operations.md,Basic Vertex Operations,DEGREE,Returns the number of incoming and outgoing ed...,"[0.018821003,0.012627394,-0.007940338,-0.00959..."
2,44,60,20,20,060-020-020-Basic_Graph_Operations.md,Basic Graph Operations,EDGES,Returns all edges in a graph. \r\n- EDGES(GRA...,"[-0.013607875,0.009249507,-0.03403819,-0.03394..."


### Connection to the HANA Vector store

Execute the following python code to create a connection to the HANA Vector storage.

In [ ]:
from hana_ml import ConnectionContext

# cc = ConnectionContext(userkey='VDB_BETA', encrypt=True)
cc= ConnectionContext(
    address='<address>',
    port='<port-number>',
    user='<username>',
    password='<password>',
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

### Creating a table

To create a table, execute the following python command.

In [ ]:
# Create a table
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE GRAPH_DOCU_QRC3_2201(ID BIGINT, L1 NVARCHAR(3), L2 NVARCHAR(3), L3 NVARCHAR(3), FILENAME NVARCHAR(100), HEADER1 NVARCHAR(5000), HEADER2 NVARCHAR(5000), TEXT NCLOB, VECTOR_STR NCLOB);'''
cursor.execute(sql_command)
cursor.close()

### Uploading the data to the database

Execute the following code to upload the data to the database.

In [ ]:
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df,
    table_name="GRAPH_DOCU_QRC3_2201", 
    allow_bigint=True,
    append=True
    )

### Creating a VECTOR column

Add a new column ```VECTOR``` to the table to store the vectors. Execute the following python code.e.

In [ ]:
# Add REAL_VECTOR column
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE GRAPH_DOCU_QRC3_2201 ADD (VECTOR REAL_VECTOR(1536));'''
cursor.execute(sql_command)
cursor.close()

### Creating vectors from strings

The vectors for the strings can be created using the ```TO_REAL_VECTOR()``` constructor. Execute the following code to update the VECTOR column with the vectors.


In [ ]:
# Create vectors from strings
cursor = cc.connection.cursor()
sql_command = '''UPDATE GRAPH_DOCU_QRC3_2201 SET VECTOR = TO_REAL_VECTOR(VECTOR_STR);'''
cursor.execute(sql_command)
cursor.close()

### Setting up hana_ml and generative-ai-hub-sdk

Import the ```hana_ml``` and ```generative-ai-hub-sdk``` packages. Set the proxy version of generative-ai-hub-sdk to ```gen-ai-hub``` for an AI Core proxy.

Execute the following python code.

In [ ]:
import hana_ml
print(hana_ml.__version__)

from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub') # for an AI Core proxy



### Get Embeddings

Define the function ```get_embedding()``` to generate embeddings for our input texts. Execute the following python code.

In [ ]:
# Get embeddings
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="text-embedding-ada-002") -> str:
    response = embeddings.create(
      model_name=model,
      input=input
    )
    return response.data[0].embedding

### Running vector search

Define a function ```run_vector_search()```. This function will search the vector database and finds the rows which are most similar to a given query.

Execute the following python code

In [ ]:
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "ID", "HEADER1", "HEADER2", "TEXT"
        FROM "GRAPH_DOCU_QRC3"
        ORDER BY "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    return df_context

Now we can test the function by sending a query. The function prints the rows that are most similar to the queries.

In [ ]:
query = "How can I run a shortest path algorithm?"
df_context = run_vector_search(query=query, metric="COSINE_SIMILARITY",k=4)
df_context

### Creating a prompt template

Create a prompt template to do retrieval augmented generation on your prompts. Execute the following python code.


In [ ]:
# Prompt. Do also use your knowledge from outside the given context.
promptTemplate_fstring = """
You are an SAP HANA Cloud expert.
You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end. 
Context:
{context}
Question:
{query}
"""
from langchain.prompts import PromptTemplate
promptTemplate = PromptTemplate.from_template(promptTemplate_fstring)

### Querying the LLM

Now create a function ```ask_llm()``` to query the LLM while using the similar vectors as context. Execute the following python code.

In [ ]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub') # for an AI Core proxy

def ask_llm(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 4) -> str:

    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        print(color.RED + 'Running retrieval augmented generation.' + color.END)
        print(color.RED + '\nEmbedding the query string and running HANA vector search.' + color.END)
        context = run_vector_search(query, metric, k)
        print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
    else:
        print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)
    prompt = promptTemplate.format(query=query, context=' '.join(df_context['TEXT'].astype('string')))
    print(color.RED + '\nAsking LLM...' + color.END)
    llm = ChatOpenAI(proxy_model_name='gpt-4', proxy_client=proxy_client)
    response = llm.invoke(prompt).content
    print(color.RED + '...completed.' + color.END)
    print(color.RED + '\nQuery: ' + color.END, query)
    print(color.BLUE + '\nResponse:' + color.BLUE)
    print(response)

Now you can test the function using a query. Run the following python code.

In [ ]:
query = "I want to calculate a shortest path. How do I do that?"

response = ask_llm(query=query, retrieval_augmented_generation=True, k=4)

# Using insert methods to write directly into the table

### Extracting data from csv file

Execute the following code to extract data from the csv file and store it in a list

In [10]:
import csv

data = []
with open('GRAPH_DOCU_2503.csv', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        try:
            data.append(row)
        except:
            print(row)

### Create a connection to hana_ml using dbapi

In [40]:

import hdbcli
from hdbcli import dbapi

cc = dbapi.connect(
    address='<address>',
    port='<port-number>',
    user='<username>',
    password='<password>',
    encrypt=True
    )


### Creating a table

To create a table, execute the following python command.

In [44]:
# Create a table
cursor = cc.cursor()
sql_command = '''CREATE TABLE TABLE10044(ID1 BIGINT, ID2 BIGINT, L1 NVARCHAR(3), L2 NVARCHAR(3), L3 NVARCHAR(3), FILENAME NVARCHAR(100), HEADER1 NVARCHAR(5000), HEADER2 NVARCHAR(5000), TEXT NCLOB, VECTOR_STR REAL_VECTOR);'''
cursor.execute(sql_command)
cursor.close()

Now we insert our data into the table we created

In [45]:
cursor = cc.cursor()
sql_insert = 'INSERT INTO TABLE10044(ID1, ID2, L1, L2, L3, FILENAME, HEADER1, HEADER2, TEXT, VECTOR_STR) VALUES (?,?,?,?,?,?,?,?,?,TO_REAL_VECTOR(?))'
cursor.executemany(sql_insert,data[1:])

(1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


### Modifying the run_vector_search function

We can modify the run_vector_search function to make use of the VECTOR_STR column for similarity search

In [68]:

cursor = cc.cursor()
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "ID2", "TEXT"
        FROM "TABLE10043"
        ORDER BY "{metric}"("VECTOR_STR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    cursor.execute(sql)
    hdf = cursor.fetchall()
    return hdf[:k]

In [69]:
query = "How can I run a shortest path algorithm?"
df_context = run_vector_search(query=query, metric="COSINE_SIMILARITY",k=4)
df_context

[(211, 'Complex GraphScript Examples', 'GraphScript Procedure Example', 'The following example depicts a more complex example of a GraphScript procedure.\nIt uses the Open Flights data set and computes the shortest path one-to-one between the airports of\nPhiladelphia and Jinan.  \n```sql\nCREATE TYPE "FLIGHTS"."TT_NODES_SPOO" AS TABLE ("NODE_KEY" NVARCHAR(100));\nCREATE TYPE "FLIGHTS"."TT_EDGES_SPOO" AS TABLE (\n"EDGE_KEY" BIGINT, "SOURCE" NVARCHAR(100), "TARGET" NVARCHAR(100));  \nCREATE OR REPLACE PROCEDURE "FLIGHTS"."GS_SPOO"(\nIN i_startNode NVARCHAR(100),         -- the ID of the start node\nIN i_endNode NVARCHAR(100),         -- the ID of the end node\nOUT o_path_length BIGINT,    -- the hop distance between start and end\nOUT o_path_weight DOUBLE,    -- the path weight/cost based on the WEIGHT attribute\nOUT o_nodes "FLIGHTS"."TT_NODES_SPOO",\nOUT o_edges "FLIGHTS"."TT_EDGES_SPOO"\n)\nLANGUAGE GRAPH READS SQL DATA AS\nBEGIN\n-- create an instance of the graph, referring to the 

### Querying the LLM

Now create a function ```ask_llm()``` to query the LLM while using the similar vectors as context. Execute the following python code.

In [80]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub') # for an AI Core proxy

def ask_llm(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 4) -> str:

    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        print(color.RED + 'Running retrieval augmented generation.' + color.END)
        print(color.RED + '\nEmbedding the query string and running HANA vector search.' + color.END)
        context = run_vector_search(query, metric, k)
        print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
    else:
        print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)
    df_context = run_vector_search(query=query, metric="COSINE_SIMILARITY",k=1)
    prompt = promptTemplate.format(query=query, context=' '.join(str(df_context)))
    print(color.RED + '\nAsking LLM...' + color.END)
    llm = ChatOpenAI(proxy_model_name='gpt-4', proxy_client=proxy_client)
    response = llm.invoke(prompt).content
    print(color.RED + '...completed.' + color.END)
    print(color.RED + '\nQuery: ' + color.END, query)
    print(color.BLUE + '\nResponse:' + color.BLUE)
    print(response)

Now you can test the function using a query. Run the following python code.

In [81]:
query = "I want to calculate a shortest path. How do I do that?"

response = ask_llm(query=query, retrieval_augmented_generation=True, k=4)

Running retrieval augmented generation.

Embedding the query string and running HANA vector search.

HANA vector search returned 4 best matching documents.

Generating LLM prompt using the context information.

Asking LLM...
...completed.

Query:  I want to calculate a shortest path. How do I do that?

Response:
To calculate the shortest path in a weighted graph, you can use the Dijkstra's algorithm in SAP HANA Cloud. Here are the steps:

1. Your graph traversal statement should use the Dijkstra's algorithm. The vertices will be visited in ascending order of their shortest distance from the start vertex.

2. To calculate the weight of each edge, you need to provide a mandatory 'WITH WEIGHT' clause.

Example:
```graphscript
TRAVERSE DIJKSTRA :g FROM :startVertex
WITH WEIGHT (EDGE e) => DOUBLE {
return :e."weight";
}
ON VISIT VERTEX (VERTEX v, DOUBLE distance) {
v."distance" = :distance;
};
```

3. In the weight function defined by 'WITH WEIGHT', the weight of the edge 'e' is calculated 